## 0. Set Paths and Imports

In [1]:
#To reset notebook
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
# SET PATHS

mainPath = 'C:/Users/Ainhoa/Documents/Postgrado/Project/'

#Path where raw data has been stored
rawDataPath = mainPath + "Data/Epochs/"

#Path where plots can be stored
pathToSavePics = mainPath + "Plots/"

pathCSV = 'C:/Users/Ainhoa/Documents/Postgrado/Project/CSV/resultsClustering_sabado9.csv'

In [3]:
# IMPORTS

import numpy as np
import scipy.io as sio
import pandas as pd
import os
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import proj3d
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.cluster.hierarchy import cophenet
from scipy.spatial.distance import pdist
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler




## 1. Load Data

In [4]:
#LOAD DATA: path where files have been stored need to be set before loading this script
def load( path ):
    #columns for the band powers
    filenames = os.listdir(path)

    #electrodes names
    electrodes = ['Fp1', 'F3', 'C3', 'Fz', 'Cz', 'Fp2', 'F4', 'C4']

    #bandPower names
    bandPower = ['Theta2+Alpha1', 'Theta', 'Alpha', 'Beta_Global', 'Beta_Alta', 'Beta_Baja', 'Gamma']

    #let us create the real dataframe
    #columns for the band powers
    columns_df=[]
    for i in electrodes:
        for j in bandPower:
            combination = i + '_(' + j + ')'
            columns_df.append(combination)

    #columns for the band power ratios
    columns_df2 = ['BPR_Fp1', 'BPR_F3', 'BPR_C3', 'BPR_Fz', 'BPR_Cz', 'BPR_Fp2', 'BPR_F4', 'BPR_C4']

    #patterns to determine experiments
    patternA = r'[\w]+A\.mat'
    patternB = r'[\w]+B\.mat'
    patternC = r'[\w]+C\.mat'

    #define the list of dataframes
    list_df=[]
    for i in filenames:
        patient = sio.loadmat(path + i)
        bandPowerDF = pd.DataFrame(patient['BandPower'].reshape(1,56), columns = columns_df)
        bandPowerRatioDF = pd.DataFrame(patient['BandPowerRatio'].reshape(1,8), columns = columns_df2)
        patientDF =  pd.concat([bandPowerDF, bandPowerRatioDF], axis=1)
        #patientDF = pd.DataFrame(patient['BandPower'].reshape(1,56), columns = columns_df)
        patientDF['patientName'] = i.split('.mat')[0]
        
        if re.match(patternA,i) != None:
            patientDF['experiment'] = 'A'
        elif re.match(patternB,i) != None:
            patientDF['experiment'] = 'B'
        elif re.match(patternC,i) != None:
            patientDF['experiment'] = 'C'
                
        list_df.append(patientDF)

    #via the concat all the dataframes are concatenated
    patientsDF = pd.concat(list_df)
    #nans are dropped
    patientsDF = patientsDF.dropna(how='any')

    #dataframe is sorted by experiment type: order A, B and C
    patientsDF.sort_values(by='experiment', ascending= True, inplace=True)

    #indexs are reset
    #patientsDF = patientsDF.reset_index(drop = True)

    #indexs are set according to the patient file
    patientsDF.set_index(['patientName'], inplace=True)
    #Return resulting data frame with all patients
    return ( patientsDF )

In [5]:
patientsDF = load ( rawDataPath )


In [6]:
def addLabel ( pathCSV, dataFrame ):
    dfClustering = pd.read_csv( pathCSV )
    dfClustering.index = dfClustering['patientName']
    result = dataFrame.reset_index().merge( dfClustering, how = 'inner').set_index('patientName')
    return ( result )

In [7]:
finalPatientsDF = addLabel ( pathCSV, patientsDF ).copy()
finalPatientsDF.head()

,Fp1_(Theta2+Alpha1),Fp1_(Theta),Fp1_(Alpha),Fp1_(Beta_Global),Fp1_(Beta_Alta),Fp1_(Beta_Baja),Fp1_(Gamma),F3_(Theta2+Alpha1),F3_(Theta),F3_(Alpha),...,BPR_C4,experiment,Unnamed: 0,cluster_GMM,cluster_Hierarchichal,cluster_Kmeans,cluster_Spectral,PCA_X,PCA_Y,Best_Cluster
patientName,,,,,,,,,,,,,,,,,,,,,
S29_A,3.603942,4.060272,2.051598,0.412040,0.188472,0.493966,0.160718,3.512098,3.725556,1.929716,...,19.688982,A,144,0,0,0,0,9.316071,2.086899,0
S74_A,2.853595,3.694020,1.676661,0.549950,0.338246,0.567536,0.306302,3.303229,3.881861,2.130401,...,29.045320,A,264,0,0,0,0,3.745638,5.466005,0
S18_A,2.376608,3.137197,1.373746,0.791417,0.446905,0.843949,0.402894,2.473590,3.208466,1.510230,...,9.950759,A,114,1,1,1,1,-0.645384,2.287421,1
S75_A,3.100145,3.696751,1.348613,0.635941,0.445743,0.553323,0.371787,3.161179,3.665984,1.415384,...,14.733004,A,267,0,0,0,0,4.036546,-1.076673,0
S119_A,2.977011,4.400604,1.274504,0.509350,0.316565,0.520461,0.289575,2.921852,3.875402,1.397294,...,8.974231,A,48,0,0,0,0,2.660355,-1.977821,0


# 1.First Mean & Std. Error over Bands& Electrodes:

In [8]:
## Calculate the mean over subjects removing the nans:
electrodes = ['Fp1', 'F3', 'C3', 'Fz', 'Cz', 'Fp2', 'F4', 'C4']
bandPower = ['Theta2+Alpha1', 'Theta', 'Alpha', 'Beta_Global', 'Beta_Alta', 'Beta_Baja', 'Gamma']
r'$\alpha > \beta$'
experiments = list( finalPatientsDF.experiment.unique() )
columnsNames = list( finalPatientsDF.columns )
lenColNames = len(columnsNames)
bandColumns = columnsNames[ 0 : lenColNames - 17]
bandPowerColumns = columnsNames[ lenColNames-10 : lenColNames - 2 ]


In [9]:
#main statistical parameters for each column
finalPatientsDF.describe()

,Fp1_(Theta2+Alpha1),Fp1_(Theta),Fp1_(Alpha),Fp1_(Beta_Global),Fp1_(Beta_Alta),Fp1_(Beta_Baja),Fp1_(Gamma),F3_(Theta2+Alpha1),F3_(Theta),F3_(Alpha),...,BPR_F4,BPR_C4,Unnamed: 0,cluster_GMM,cluster_Hierarchichal,cluster_Kmeans,cluster_Spectral,PCA_X,PCA_Y,Best_Cluster
count,334.000000,334.000000,334.000000,334.000000,334.000000,334.000000,334.000000,334.000000,334.000000,334.000000,...,334.000000,334.000000,334.000000,334.000000,334.000000,334.00000,334.000000,3.340000e+02,3.340000e+02,334.000000
mean,2.483312,3.217692,1.671830,0.651677,0.416584,0.661549,0.373267,2.630579,3.404363,1.772783,...,10.242191,15.432172,166.500000,0.565868,0.682635,0.50000,0.592814,-1.170481e-13,7.837643e-14,0.565868
std,0.782261,0.802305,0.607621,0.207101,0.196546,0.173012,0.178919,0.788942,0.802393,0.634528,...,7.533514,14.344398,96.561725,0.496386,0.466149,0.50075,0.492047,5.265490e+00,3.008563e+00,0.496386
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,-2.035912e+01,-8.405420e+00,0.000000
25%,1.916719,2.628290,1.250038,0.502963,0.265491,0.538670,0.231347,2.118802,2.907087,1.306067,...,5.024918,6.907607,83.250000,0.000000,0.000000,0.00000,0.000000,-3.486862e+00,-1.997602e+00,0.000000
50%,2.378577,3.218909,1.510521,0.632298,0.383958,0.643110,0.338936,2.503656,3.433101,1.634434,...,7.870704,10.494277,166.500000,1.000000,1.000000,0.50000,1.000000,-7.543384e-02,-7.996628e-02,1.000000
75%,2.944680,3.758272,1.997923,0.773415,0.525750,0.762751,0.467849,3.077061,3.942839,2.089760,...,13.417129,19.619021,249.750000,1.000000,1.000000,1.00000,1.000000,3.907208e+00,2.085553e+00,1.000000
max,5.488992,5.364788,3.722925,1.505944,1.203910,1.349076,1.070237,5.839428,5.332177,3.956982,...,49.600342,122.647030,333.000000,1.000000,1.000000,1.00000,1.000000,1.393875e+01,8.728430e+00,1.000000


# Mean Power for each electrode:

In [10]:
# Plot the mean over tetrodes
meanExpA = finalPatientsDF.loc[ finalPatientsDF.experiment == 'A'].mean(axis=0)
stdExpA = finalPatientsDF.loc[ finalPatientsDF.experiment == 'A' ].std(axis=0) /np.sqrt(len(finalPatientsDF.loc[ finalPatientsDF.experiment == 'A']))
meanExpB = finalPatientsDF.loc[ finalPatientsDF.experiment == 'B'].mean(axis=0)
stdExpB = finalPatientsDF.loc[ finalPatientsDF.experiment == 'B' ].std(axis=0)/np.sqrt(len(finalPatientsDF.loc[ finalPatientsDF.experiment == 'B']))
meanExpC = finalPatientsDF.loc[ finalPatientsDF.experiment == 'C'].mean(axis=0)
stdExpC = finalPatientsDF.loc[ finalPatientsDF.experiment == 'C' ].std(axis=0) /np.sqrt(len(finalPatientsDF.loc[ finalPatientsDF.experiment == 'C']))
z= 1;
fig = plt.figure()
fig.add_axes([0,0,1,1])

for i in range(0, len(bandColumns)- 1, 7): #over electrodes
    electrodePossition = int((i+1)/7)
    electrodeBand =  bandColumns[i:i+7]
    ax = plt.subplot(2,4,z)

    plt.plot(np.arange(0,7), meanExpA[i:i+7], 'k', color='#3F7F4C',label = 'ExpA')
    plt.plot(np.arange(0,7), meanExpB[i:i+7], 'k', color='#CC4F1B',label = 'ExpB')
    plt.plot(np.arange(0,7), meanExpC[i:i+7], 'k', color='#1B2ACC',label = 'ExpC')

    plt.fill_between(np.arange(0,7), meanExpA[i:i+7]-stdExpA[i:i+7], meanExpA[i:i+7]+stdExpA[i:i+7],
        alpha=0.5, edgecolor='#3F7F4C', facecolor='#7EFF99',linewidth=0)    
    plt.fill_between(np.arange(0,7), meanExpB[i:i+7]-stdExpB[i:i+7], meanExpB[i:i+7]+stdExpB[i:i+7],
        alpha=0.5, edgecolor='#CC4F1B', facecolor='#FF9848',linewidth=0)
    plt.fill_between(np.arange(0,7), meanExpC[i:i+7]-stdExpC[i:i+7], meanExpC[i:i+7]+stdExpC[i:i+7],
        alpha=0.2, edgecolor='#1B2ACC', facecolor='#089FFF',linewidth=0)
    plt.xticks(range(0,7),bandPower , color='black',rotation=40,horizontalalignment='center',verticalalignment='top')
    plt.ylabel('Mean Power')
    plt.yticks(np.arange(0, 5, 1.0))  
    ax.set_ylim([0, 5])

    if z == 8:
        plt.legend()    
    ax.text(.5,.9,str(electrodes[z-1]),
        horizontalalignment='center',
        transform=ax.transAxes,fontsize=20, color='black',fontweight='bold')
    z=z+1; 

    
figManager = plt.get_current_fig_manager()
figManager.window.showMaximized()
picPath = pathToSavePics + "MeanPower.pdf"
plt.savefig( picPath )
plt.show()
        


# Mean For each of the clustering groups


In [11]:
for exp in experiments:
    indexExpk0 = (finalPatientsDF.loc[ (finalPatientsDF.experiment == exp) & (finalPatientsDF.cluster_Hierarchichal == 0)]).iloc[:,0:56]
    indexExpk1 = (finalPatientsDF.loc[ (finalPatientsDF.experiment == exp) & (finalPatientsDF.cluster_Hierarchichal == 1)]).iloc[:,0:56]
    # Do the mean of the subset of subjects:
    meanExpk0 = indexExpk0.mean(axis=0)
    stdExpk0 = indexExpk0.std(axis=0) /np.sqrt(len(indexExpk0))
    meanExpk1 = indexExpk1.mean(axis=0)
    stdExpk1 = indexExpk1.std(axis=0) /np.sqrt(len(indexExpk1))
    z= 1;
    fig = plt.figure()
    fig.add_axes([0,0,1,1])
    for i in range(0, len(bandColumns)- 1, 7): #over electrodes
        electrodePossition = int((i+1)/7)
        electrodeBand =  bandColumns[i:i+7]
        ax=plt.subplot(2,4,z)    
        plt.plot(np.arange(0,7), meanExpk0[i:i+7], 'k', color='green',label = 'k0')
        plt.plot(np.arange(0,7), meanExpk1[i:i+7], 'k', color='red',label = 'k1')    
        plt.fill_between(np.arange(0,7), meanExpk0[i:i+7]-stdExpk0[i:i+7], meanExpk0[i:i+7]+stdExpk0[i:i+7],
            alpha=0.3, edgecolor='green', facecolor='green',linewidth=0)    
        plt.fill_between(np.arange(0,7), meanExpk1[i:i+7]-stdExpk1[i:i+7], meanExpk1[i:i+7]+stdExpk1[i:i+7],
            alpha=0.3, edgecolor='red', facecolor='red',linewidth=0)
        if z== 5 or z== 6 or z== 7 or z== 8:
            plt.xticks(range(0,7),bandPower , color='black',rotation=40,horizontalalignment='center')
            plt.xlabel('Band')
        else:
            plt.xticks(range(0,7),[])
        if z== 1 or z== 5:
            plt.ylabel('Mean Power')
            plt.yticks(np.arange(0, 5, 1.0))
        else:  
            plt.yticks(range(0,5),[])
        ax.set_ylim([0, 5])

        if z == 8:
            plt.legend()
        ax.text(.5,.9,str(electrodes[z-1]),
            horizontalalignment='center',
            transform=ax.transAxes,fontsize=20, color='black',fontweight='bold')
        z=z+1;
    fig.suptitle('Experiment '+exp+'cluster_Hierarchichal')
    figManager = plt.get_current_fig_manager()
    figManager.window.showMaximized()
    picPath = pathToSavePics + "MeanPowerClusteringExp"+exp+".pdf"
    plt.savefig( picPath )
    plt.show()   

In [415]:
z= 1;
fig = plt.figure()
fig.add_axes([0,0,1,1])
for exp in experiments:
    indexExpk0 = (finalPatientsDF.loc[ (finalPatientsDF.experiment == exp) & (finalPatientsDF.cluster_Hierarchichal == 0)]).iloc[:,56:64]
    indexExpk1 = (finalPatientsDF.loc[ (finalPatientsDF.experiment == exp) & (finalPatientsDF.cluster_Hierarchichal == 1)]).iloc[:,56:64]
    # Do the mean of the subset of subjects:
    meanExpk0 = indexExpk0.mean(axis=0)
    stdExpk0 = indexExpk0.std(axis=0) /np.sqrt(len(indexExpk0))
    meanExpk1 = indexExpk1.mean(axis=0)
    stdExpk1 = indexExpk1.std(axis=0) /np.sqrt(len(indexExpk1))
    
    ax=plt.subplot(1,3,z) 
    plt.plot(np.arange(0,7), meanExpk0[0:7], 'k', color='green',label = 'k0')
    plt.plot(np.arange(0,7), meanExpk1[0:7], 'k', color='red',label = 'k1')    
    plt.fill_between(np.arange(0,7), meanExpk0[0:7]-stdExpk0[0:7], meanExpk0[0:7]+stdExpk0[0:7],
        alpha=0.3, edgecolor='green', facecolor='green',linewidth=0)    
    plt.fill_between(np.arange(0,7), meanExpk1[0:7]-stdExpk1[0:7], meanExpk1[0:7]+stdExpk1[0:7],
        alpha=0.3, edgecolor='red', facecolor='red',linewidth=0)
   
    if z == 1:
        plt.legend()
    ax.text(.5,.9,'Power Ratio',
        horizontalalignment='center',
        transform=ax.transAxes,fontsize=20, color='black',fontweight='bold')
    z=z+1;
fig.suptitle('Experiment '+exp+'cluster_Hierarchichal')
#figManager = plt.get_current_fig_manager()
#figManager.window.showMaximized()
#picPath = pathToSavePics + "MeanPowerClusteringExp"+exp+".pdf"
#plt.savefig( picPath )
plt.show()   

# Subjects ordered by the peak of theta in each electrode


In [503]:
# Plot 3D surface patients
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

for exp in experiments:
    SelectExp = finalPatientsDF.loc[ finalPatientsDF.experiment == exp]
    xgv = np.arange(0,7) #all the bands
    ygv = np.arange(0,len(SelectExp)) #all subjects
    [X,Y] = np.meshgrid(xgv, ygv) 
    fig = plt.figure()
    p=1
    for i in range(0, len(bandColumns)- 1, 7): #over electrodes
        ax = fig.add_subplot(2,4,p, projection='3d')
        dfExp = SelectExp.sort_values(by=list(SelectExp.columns.values)[i+1], ascending=0) # reorder by the peak of theta
        ax.plot_surface(X, Y, dfExp.ix[:,i:i+7], rstride=1, cstride=1, cmap=plt.cm.jet, linewidth=0.25)
        plt.xticks(range(0,7),bandPower , color='black',rotation=40,horizontalalignment='right')
        plt.title('Band Power');
        plt.ylabel('Subjects')
        plt.xlabel('Band')
        plt.title(str(electrodes[p-1]),horizontalalignment='center',verticalalignment='center',fontsize=20, color='black',fontweight='bold')
        p=p+1
        ax.set_zlim([0, 6])

    fig.suptitle('Experiment '+exp)
    figManager = plt.get_current_fig_manager()
    figManager.window.showMaximized()
    picPath = pathToSavePics + "BandPowerAllSubjectsExp"+exp+"3D.pdf"
    plt.savefig( picPath )
    plt.show()


In [368]:

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from scipy.special import jn
from matplotlib import cm
from matplotlib.colors import LightSource
# Set view parameters for all subplots.
azimuth = 45
altitude = 60
# Create a light source object for light from
# 0 degrees azimuth, 0 degrees elevation.
light = LightSource(0, 0)


for exp in experiments:
    SelectExp = finalPatientsDF.loc[ finalPatientsDF.experiment == exp]
    xgv = np.arange(0,7) #all the bands
    ygv = np.arange(0,len(SelectExp)) #all subjects
    [X,Y] = np.meshgrid(xgv, ygv) 
    fig = plt.figure()
    p=1
    for i in range(0, len(bandColumns)- 1, 7): #over electrodes
        ax = fig.add_subplot(2,4,p, projection='3d')
        dfExp = SelectExp.sort_values(by=list(SelectExp.columns.values)[i+1], ascending=0) # reorder by the peak of theta
        white = np.ones((dfExp.ix[:,i:i+7].shape[0], dfExp.ix[:,i:i+7].shape[1], 3))
        red = white * np.array([1,0,0])
        green = white * np.array([0,1,0])
        blue = white * np.array([0,0,1])
        black = white * np.array([0,0,0])
        illuminated_surface = light.shade_rgb(blue, dfExp.ix[:,i:i+7])
        ax.plot_surface(X, Y, dfExp.ix[:,i:i+7], rstride=1, cstride=1, linewidth=0,
                antialiased=False, facecolors=illuminated_surface)
        #ax.plot_surface(X, Y, dfExp.ix[:,i:i+7], rstride=1, cstride=1, cmap=plt.cm.jet, linewidth=0.25)
        plt.xticks(range(0,7),bandPower , color='black',rotation=40,horizontalalignment='right')
        plt.title('Band Power');
        plt.ylabel('Subjects')
        plt.xlabel('Band')
        plt.title(str(electrodes[p-1]),horizontalalignment='center',verticalalignment='center',fontsize=20, color='black',fontweight='bold')
        p=p+1
        ax.set_zlim([0, 6])
    fig.suptitle('Experiment '+exp)
    figManager = plt.get_current_fig_manager()
    figManager.window.showMaximized()
    #picPath = pathToSavePics + "BandPowerAllSubjectsExp"+exp+"3D.pdf"
    #plt.savefig( picPath )
    plt.show()





In [369]:
# Plot 2D surface patients
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

for exp in experiments:
    SelectExp = finalPatientsDF.loc[ finalPatientsDF.experiment == exp]
    xgv = np.arange(0,7) #all the bands
    ygv = np.arange(0,len(SelectExp)) #all subjects
    [X,Y] = np.meshgrid(xgv, ygv) 
    fig = plt.figure()
    p=1
    for i in range(0, len(bandColumns)- 1, 7): #over electrodes
        ax = fig.add_subplot(2,4,p)
        dfExp = SelectExp.sort_values(by=list(SelectExp.columns.values)[i+1], ascending=0) # reorder by the peak of theta
        v = np.linspace(0, 6, 15, endpoint=True)
        cax = ax.contourf(X, Y, dfExp.ix[:,i:i+7],v,cmap=plt.cm.jet)
        plt.xticks(range(0,7),bandPower , color='black',rotation=40,horizontalalignment='right')
        plt.ylabel('Subjects')
        plt.xlabel('Band')
        ax.text(.5,.9,str(electrodes[p-1]),
            horizontalalignment='center',
            transform=ax.transAxes,fontsize=20, color='black',fontweight='bold')
        if p == 8:
            cbar = fig.colorbar(cax)
        p=p+1
    fig.suptitle('Experiment '+exp)
    figManager = plt.get_current_fig_manager()
    figManager.window.showMaximized()
    picPath = pathToSavePics + "BandPowerAllSubjectsExp"+exp+"2D.pdf"
    plt.savefig( picPath )
    plt.show()

# 3d surface of the mean of subjects


In [370]:
# Plot 3D surface
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
xgv = np.arange(0,7) #electrodes
ygv = np.arange(0,8)
[X,Y] = np.meshgrid(xgv, ygv)

fig = plt.figure()
k = 1
for exp in experiments:
    meanExp = finalPatientsDF.loc[ finalPatientsDF.experiment == exp].mean(axis=0)
    stdExp = finalPatientsDF.loc[ finalPatientsDF.experiment == exp ].std(axis=0) /np.sqrt(len(finalPatientsDF.loc[ finalPatientsDF.experiment == exp]))
    VExp = meanExp[0:56].values.reshape(meanExp[0:56].shape[0]/7,7)
    ax = fig.add_subplot(1,3,k, projection='3d')
    ax.plot_surface(X, Y, VExp, rstride=1, cstride=1, cmap=plt.cm.jet, linewidth=0.25)
    plt.yticks(range(0,8),electrodes , color='black',rotation=0,horizontalalignment='left')
    plt.xticks(range(0,7),bandPower , color='black',rotation=40,horizontalalignment='right')
    plt.title('Exp '+exp);
    plt.ylabel('Electrodes')
    plt.xlabel('Band')
    k=k+1
    ax.set_zlim([0, 5])
    
figManager = plt.get_current_fig_manager()
figManager.window.showMaximized()
picPath = pathToSavePics + "MeanBandPower"+"3D.pdf"
plt.savefig( picPath )
plt.show()


C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:13: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


# 3d surface of the mean of subjects with clustering

In [371]:
# Plot 3D surface
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
xgv = np.arange(0,7) #electrodes
ygv = np.arange(0,8)
[X,Y] = np.meshgrid(xgv, ygv)

fig = plt.figure()
k = 1
for exp in 'experiments':
    indexExpk0 = (finalPatientsDF.loc[ (finalPatientsDF.experiment == exp) & (finalPatientsDF.cluster_Hierarchichal == 0)]).iloc[:,0:56]
    indexExpk1 = (finalPatientsDF.loc[ (finalPatientsDF.experiment == exp) & (finalPatientsDF.cluster_Hierarchichal == 1)]).iloc[:,0:56]
    # Do the mean of the subset of subjects:
    meanExpk0 = indexExpk0.mean(axis=0)
    stdExpk0 = indexExpk0.std(axis=0) /np.sqrt(len(indexExpk0))
    meanExpk1 = indexExpk1.mean(axis=0)
    stdExpk1 = indexExpk1.std(axis=0) /np.sqrt(len(indexExpk1))
    
    VExpk0 = meanExpk0[0:56].values.reshape(meanExpk0[0:56].shape[0]/7,7)
    VExpk1 = meanExpk1[0:56].values.reshape(meanExpk1[0:56].shape[0]/7,7)

    ax = fig.add_subplot(2,3,k, projection='3d')
    ax.plot_surface(X, Y, VExpk0, rstride=1, cstride=1, cmap=plt.cm.jet, linewidth=0.25)
    plt.yticks(range(0,8),electrodes , color='black',rotation=0,horizontalalignment='left')
    plt.xticks(range(0,7),bandPower , color='black',rotation=40,horizontalalignment='right')
    ax.set_zlim([0, 5])
    plt.title('Exp '+exp+'Subgroup 0');
    plt.ylabel('Electrodes')
    plt.xlabel('Band')

    ax = fig.add_subplot(2,3,k+3, projection='3d')
    ax.plot_surface(X, Y, VExpk1, rstride=1, cstride=1, cmap=plt.cm.jet, linewidth=0.25)
    plt.yticks(range(0,8),electrodes , color='black',rotation=0,horizontalalignment='left')
    plt.xticks(range(0,7),bandPower , color='black',rotation=40,horizontalalignment='right')
    plt.title('Exp '+exp+' Subgroup 1');        
    ax.set_zlim([0, 5])
    plt.ylabel('Electrodes')
    plt.xlabel('Band')
    k=k+1
fig.suptitle(list(ClusteringdF.columns.values)[ClusteringType])
figManager = plt.get_current_fig_manager()
figManager.window.showMaximized()
picPath = pathToSavePics + "MeanBandPowerClustering"+"3D.pdf"
plt.savefig( picPath )
plt.show()

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:19: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:20: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


ValueError: num must be 1 <= num <= 6, not 7

# 2d surface of the mean of subjects


In [373]:
# Plot 2D surface
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
xgv = np.arange(0,7) #electrodes
ygv = np.arange(0,8)
[X,Y] = np.meshgrid(xgv, ygv)
k=1
fig = plt.figure()
for exp in experiments:
    meanExp = finalPatientsDF.loc[ finalPatientsDF.experiment == exp].mean(axis=0)    
    VExp = meanExp[0:56].values.reshape(meanExp[0:56].shape[0]/7,7)
    ax = fig.add_subplot(1,3,k)
    v = np.linspace(0, 4.5, 15, endpoint=True)
    cax = ax.contourf(X, Y, VExp,v,cmap=plt.cm.jet)
    if exp == 'C':
        cbar = fig.colorbar(cax)
    ax.set_xlabel('Band'); ax.set_ylabel('Electrodes'); ax.set_title('Exp '+exp)
    plt.yticks(range(0,8),electrodes , color='black')
    plt.xticks(range(0,7),bandPower , color='black',rotation=40,horizontalalignment='right')
    k=k+1
figManager = plt.get_current_fig_manager()
figManager.window.showMaximized()
picPath = pathToSavePics + "MeanBandPower"+"2D.pdf"
plt.savefig( picPath )
plt.show()

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:11: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


# 2d surface of the pecentage of the difference  mean of subjects after clustering

In [493]:
from __future__ import print_function
from __future__ import division
from __future__ import absolute_import

import scipy.interpolate
import numpy
import matplotlib 
import matplotlib.pyplot as plt

# close old plots
plt.close("all")

# some parameters
N = 300             # number of points for interpolation
xy_center = [2,2]   # center of the plot
radius = 2          # radius
for exp in experiments:
    # mostly original code
    indexExpk0 = (finalPatientsDF.loc[ (finalPatientsDF.experiment == exp) & (finalPatientsDF.cluster_Hierarchichal == 0)]).iloc[:,0:56]
    indexExpk1 = (finalPatientsDF.loc[ (finalPatientsDF.experiment == exp) & (finalPatientsDF.cluster_Hierarchichal == 1)]).iloc[:,0:56]
    # Do the mean of the subset of subjects:
    difEpx = 100*(indexExpk0.mean(axis=0)-indexExpk1.mean(axis=0))/indexExpk1.mean(axis=0)
    VExp = difEpx[0:56].values.reshape(difEpx[0:56].shape[0]/7,7)

    m=0#np.amin(difEpx[0:56].values)
    ma=np.amax(difEpx[0:56].values)
    meanRMap1 = [m, m]
    meanRMap2 = [m,  m,  m,  m , m, m, m, m, m, m, m, m, m]

    koordElectrodes = [[1.5,3.7],[2.5,3.7],[1.3,3],[2.7,3],[2,3],[1,2],[3,2],[2,2]]
    koordMap = [[1,4],[3,4],[1.5,3.7],[2.5,3.7],[1.3,3],[2.7,3],[2,3],[1,2],[3,2],[2,2],[1,1],[3,1],[2,1],[1,0],
                [3,0],[0,3],[4,3],[0.6,2],[3.4,2],[0,1],[4,1],[1,3],[3,3]]
    #For map
    x,y = [],[]
    for i in koordMap:
        x.append(i[0])
        y.append(i[1])
    #For electrodes:    
    xElectrodes,yElectrodes = [],[]
    for i in koordElectrodes:
        xElectrodes.append(i[0])
        yElectrodes.append(i[1])  


    for iBand in range(0,7,1): #over electrodes

        meanRMap=np.concatenate((meanRMap1,VExp[:,iBand],meanRMap2), axis=0)
        z = meanRMap
        xi = numpy.linspace(-2, 5, N)
        yi = numpy.linspace(-2, 5, N)
        zi = scipy.interpolate.griddata((x, y), z, (xi[None,:], yi[:,None]), method='cubic')

        # set points > radius to not-a-number. They will not be plotted.
        # the dr/2 makes the edges a bit smoother
        dr = xi[1] - xi[0]
        for i in range(N):
            for j in range(N):
                r = numpy.sqrt((xi[i] - xy_center[0])**2 + (yi[j] - xy_center[1])**2)
                if (r - dr/2) > radius:
                    zi[j,i] = "nan"

        # set aspect = 1 to make it a circle
        # make figure
        fig = plt.figure()
        ax = fig.add_subplot(111, aspect = 1)
        # draw a circle
        circle = matplotlib.patches.Circle(xy = xy_center, radius = radius, edgecolor = "k", facecolor = "lightgreen")
        ax.add_patch(circle)
        # use different number of levels for the fill and the lines
        v = np.linspace(-120, 120, 60, endpoint=True)
        CS = ax.contourf(xi, yi, zi, v, cmap = plt.cm.jet, zorder = 2)
        ax.contour(xi, yi, zi, 15, colors = "grey", zorder = 2)
        # make a color bar   
        cbar = fig.colorbar(CS, ax=ax)        
        # add the data points
        # I guess there are no data points outside the head...
        ax.scatter(xElectrodes, yElectrodes, marker = 'o', c = 'k', s = 30, zorder = 3)
        # make the axis invisible 
        for loc, spine in ax.spines.iteritems():
            # use ax.spines.items() in Python 3
            spine.set_linewidth(0)

        # remove the ticks
        ax.set_xticks([])
        ax.set_yticks([])

        # Add some body parts. Hide unwanted parts by setting the zorder low
        # add two ears
        circle = matplotlib.patches.Ellipse(xy = [0,2], width = 0.5, height = 1.0, angle = 0, edgecolor = "k", facecolor = "w", zorder = 0)
        ax.add_patch(circle)
        circle = matplotlib.patches.Ellipse(xy = [4,2], width = 0.5, height = 1.0, angle = 0, edgecolor = "k", facecolor = "w", zorder = 0)
        ax.add_patch(circle)
        # add a nose
        xy = [[1.5,3], [2,4.5],[2.5,3]]
        polygon = matplotlib.patches.Polygon(xy = xy, facecolor = "w", zorder = 0)
        ax.add_patch(polygon) 
        plt.title(bandPower[iBand]+ ' Exp'+exp);   
        # set axes limits
        ax.set_xlim(-0.5, 4.5)
        ax.set_ylim(-0.5, 4.5)
        picPath = pathToSavePics + "MeanBandDifferencePercentage"+bandPower[iBand]+"Exp"+exp+".pdf"
        plt.savefig( picPath )
    plt.show() 

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:23: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


# # 2d surface of the percentage of the difference mean Ratio of subjects after clustering

In [ ]:
indexExpk0 = (finalPatientsDF.loc[ (finalPatientsDF.experiment == exp) & (finalPatientsDF.cluster_Hierarchichal == 0)]).iloc[:,56:64]
indexExpk0

In [500]:
from __future__ import print_function
from __future__ import division
from __future__ import absolute_import

import scipy.interpolate
import numpy
import matplotlib 
import matplotlib.pyplot as plt

# close old plots
plt.close("all")

# some parameters
N = 300             # number of points for interpolation
xy_center = [2,2]   # center of the plot
radius = 2          # radius
for exp in experiments:
    # mostly original code
    indexExpk0 = (finalPatientsDF.loc[ (finalPatientsDF.experiment == exp) & (finalPatientsDF.cluster_Hierarchichal == 0)]).iloc[:,56:64]
    indexExpk1 = (finalPatientsDF.loc[ (finalPatientsDF.experiment == exp) & (finalPatientsDF.cluster_Hierarchichal == 1)]).iloc[:,56:64]
    # Do the mean of the subset of subjects:
    VExp = 100*(indexExpk0.mean(axis=0)-indexExpk1.mean(axis=0))/indexExpk1.mean(axis=0)


    m=0#np.amin(difEpx[0:56].values)
    ma=np.amax(VExp.values)
    meanRMap1 = [m, m]
    meanRMap2 = [m,  m,  m,  m , m, m, m, m, m, m, m, m, m]

    koordElectrodes = [[1.5,3.7],[2.5,3.7],[1.3,3],[2.7,3],[2,3],[1,2],[3,2],[2,2]]
    koordMap = [[1,4],[3,4],[1.5,3.7],[2.5,3.7],[1.3,3],[2.7,3],[2,3],[1,2],[3,2],[2,2],[1,1],[3,1],[2,1],[1,0],
                [3,0],[0,3],[4,3],[0.6,2],[3.4,2],[0,1],[4,1],[1,3],[3,3]]
    #For map
    x,y = [],[]
    for i in koordMap:
        x.append(i[0])
        y.append(i[1])
    #For electrodes:    
    xElectrodes,yElectrodes = [],[]
    for i in koordElectrodes:
        xElectrodes.append(i[0])
        yElectrodes.append(i[1])  

    meanRMap=np.concatenate((meanRMap1,VExp,meanRMap2), axis=0)
    z = meanRMap
    xi = numpy.linspace(-2, 5, N)
    yi = numpy.linspace(-2, 5, N)
    zi = scipy.interpolate.griddata((x, y), z, (xi[None,:], yi[:,None]), method='cubic')

    # set points > radius to not-a-number. They will not be plotted.
    # the dr/2 makes the edges a bit smoother
    dr = xi[1] - xi[0]
    for i in range(N):
        for j in range(N):
            r = numpy.sqrt((xi[i] - xy_center[0])**2 + (yi[j] - xy_center[1])**2)
            if (r - dr/2) > radius:
                zi[j,i] = "nan"

    # set aspect = 1 to make it a circle
    # make figure
    fig = plt.figure()
    ax = fig.add_subplot(111, aspect = 1)
    # draw a circle
    circle = matplotlib.patches.Circle(xy = xy_center, radius = radius, edgecolor = "k", facecolor = "lightgreen")
    ax.add_patch(circle)
    # use different number of levels for the fill and the lines
    v = np.linspace(-180, 180, 60, endpoint=True)
    CS = ax.contourf(xi, yi, zi, v, cmap = plt.cm.jet, zorder = 2)
    ax.contour(xi, yi, zi, 15, colors = "grey", zorder = 2)
    # make a color bar   
    cbar = fig.colorbar(CS, ax=ax)        
    # add the data points
    # I guess there are no data points outside the head...
    ax.scatter(xElectrodes, yElectrodes, marker = 'o', c = 'k', s = 30, zorder = 3)
    # make the axis invisible 
    for loc, spine in ax.spines.iteritems():
        # use ax.spines.items() in Python 3
        spine.set_linewidth(0)

    # remove the ticks
    ax.set_xticks([])
    ax.set_yticks([])

    # Add some body parts. Hide unwanted parts by setting the zorder low
    # add two ears
    circle = matplotlib.patches.Ellipse(xy = [0,2], width = 0.5, height = 1.0, angle = 0, edgecolor = "k", facecolor = "w", zorder = 0)
    ax.add_patch(circle)
    circle = matplotlib.patches.Ellipse(xy = [4,2], width = 0.5, height = 1.0, angle = 0, edgecolor = "k", facecolor = "w", zorder = 0)
    ax.add_patch(circle)
    # add a nose
    xy = [[1.5,3], [2,4.5],[2.5,3]]
    polygon = matplotlib.patches.Polygon(xy = xy, facecolor = "w", zorder = 0)
    ax.add_patch(polygon) 
    plt.title('Power Ratio Exp '+exp);   
    # set axes limits
    ax.set_xlim(-0.5, 4.5)
    ax.set_ylim(-0.5, 4.5)
    picPath = pathToSavePics + "MeanRatioBandDifferencePercentageExp"+exp+".pdf"
    plt.savefig( picPath )
plt.show() 

# 2d surface of the mean of subjects after clustering


In [501]:
# Plot 2D surface
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
xgv = np.arange(0,7) #electrodes
ygv = np.arange(0,8)
[X,Y] = np.meshgrid(xgv, ygv)
k=1
fig = plt.figure()
for exp in experiments:
    meanExp = finalPatientsDF.loc[ finalPatientsDF.experiment == exp].mean(axis=0)    
    VExp = meanExp[0:56].values.reshape(meanExp[0:56].shape[0]/7,7)
    ax = fig.add_subplot(3,3,k)
    v = np.linspace(0, 4.5, 15, endpoint=True)
    cax = ax.contourf(X, Y, VExp,v,cmap=plt.cm.jet)
    if exp == 'C':
        cbar = fig.colorbar(cax)
    if exp == 'A':    
        plt.yticks(range(0,8),electrodes , color='black') 

    else :  
        plt.yticks(range(0,8),[] , color='black')
    ax.set_title('Exp '+exp,fontsize=15, color='black',fontweight='bold')    
    plt.xticks(range(0,7),[])
    ax.text(.5,.9,'Total',horizontalalignment='center',transform=ax.transAxes,fontsize=15, color='black')

    k=k+1

# Plot clustering 

xgv = np.arange(0,7) #electrodes
ygv = np.arange(0,8)
[X,Y] = np.meshgrid(xgv, ygv)
k=1
for exp in experiments:
    indexExpk0 = (finalPatientsDF.loc[ (finalPatientsDF.experiment == exp) & (finalPatientsDF.cluster_Hierarchichal == 0)]).iloc[:,0:56]
    indexExpk1 = (finalPatientsDF.loc[ (finalPatientsDF.experiment == exp) & (finalPatientsDF.cluster_Hierarchichal == 1)]).iloc[:,0:56]
    # Do the mean of the subset of subjects:
    meanExpk0 = indexExpk0.mean(axis=0)
    stdExpk0 = indexExpk0.std(axis=0) /np.sqrt(len(indexExpk0))
    meanExpk1 = indexExpk1.mean(axis=0)
    stdExpk1 = indexExpk1.std(axis=0) /np.sqrt(len(indexExpk1))
    
    VExpk0 = meanExpk0[0:56].values.reshape(meanExpk0[0:56].shape[0]/7,7)
    VExpk1 = meanExpk1[0:56].values.reshape(meanExpk1[0:56].shape[0]/7,7)
    
    ax = fig.add_subplot(3,3,k+3)
    v = np.linspace(0, 4.5, 15, endpoint=True)
    cax = ax.contourf(X, Y, VExpk0,v,cmap=plt.cm.jet)
    if exp == 'A':    
        plt.yticks(range(0,8),electrodes , color='black')
        ax.set_ylabel('Electrodes',fontsize=15, color='black');
    else :  
        plt.yticks(range(0,8),[] , color='black')        
    plt.xticks(range(0,7),[] )
    ax.text(.5,.9,'k0',horizontalalignment='center',transform=ax.transAxes,fontsize=15, color='black')

    ax = fig.add_subplot(3,3,k+6)
    v = np.linspace(0, 4.5, 15, endpoint=True)
    cax = ax.contourf(X, Y, VExpk1,v,cmap=plt.cm.jet)
    ax.set_xlabel('Band',fontsize=15, color='black');
    if exp == 'A':    
        plt.yticks(range(0,8),electrodes , color='black')
        #ax.set_ylabel('Electrodes'); 
    else :  
        plt.yticks(range(0,8),[] , color='black')        
    ax.text(.5,.9,'k1',horizontalalignment='center',transform=ax.transAxes,fontsize=15, color='black')
    plt.xticks(range(0,7),bandPower , color='black',rotation=40,horizontalalignment='right')
    k=k+1
fig.suptitle('Band Power',fontsize=20, color='black',fontweight='bold')
figManager = plt.get_current_fig_manager()
figManager.window.showMaximized()
picPath = pathToSavePics + "MeanBandPowerTotal&Clustering"+"2D.pdf"
plt.savefig( picPath )
plt.show()

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:11: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:43: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:44: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [502]:
# Plot 2D surface
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
# Plot clustering 
xgv = np.arange(0,7) #electrodes
ygv = np.arange(0,8)
[X,Y] = np.meshgrid(xgv, ygv)
k=1
fig = plt.figure()
for exp in experiments:
    indexExpk0 = (finalPatientsDF.loc[ (finalPatientsDF.experiment == exp) & (finalPatientsDF.cluster_Hierarchichal == 0)]).iloc[:,0:56]
    indexExpk1 = (finalPatientsDF.loc[ (finalPatientsDF.experiment == exp) & (finalPatientsDF.cluster_Hierarchichal == 1)]).iloc[:,0:56]
    # Do the mean of the subset of subjects:
    difEpx = 100*(indexExpk0.mean(axis=0)-indexExpk1.mean(axis=0))/indexExpk1.mean(axis=0)
    VExp = difEpx[0:56].values.reshape(difEpx[0:56].shape[0]/7,7)
    ax = fig.add_subplot(1,3,k)
    v = np.linspace(-90, 90, 60, endpoint=True)
    cax = ax.contourf(X, Y, VExp,v,cmap=plt.cm.jet)
    if exp == 'C':  
        cbar = fig.colorbar(cax, ax=ax)
    if exp == 'A':    
        plt.yticks(range(0,8),electrodes , color='black')
        ax.set_ylabel('Electrodes',fontsize=15, color='black');
    else :  
        plt.yticks(range(0,8),[] , color='black')        
    plt.xticks(range(0,7),bandPower , color='black',rotation=40,horizontalalignment='right')
    k=k+1
fig.suptitle('Difference %',fontsize=20, color='black',fontweight='bold')
figManager = plt.get_current_fig_manager()
figManager.window.showMaximized()
picPath = pathToSavePics + "MeanBandDifferencePercentageAllExp2D.pdf"
plt.savefig( picPath )
plt.show()

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:15: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


# Plot correlations in one electrode between bands:


In [420]:
#Plot correlations in one electrode between bands:

electrodes = ['Fp1', 'F3', 'C3', 'Fz', 'Cz', 'Fp2', 'F4', 'C4']

from string import letters
sns.set(style="white")
for exp in experiments:
    SelectExp = finalPatientsDF.loc[ finalPatientsDF.experiment == exp]
    j = 1;
    for i in range(0, len(bandColumns)- 1, 7): #over electrodes
        # Compute the correlation matrix
        corr = SelectExp.ix[:,i:i+7].corr()
        # Generate a mask for the upper triangle
        mask = np.zeros_like(corr, dtype=np.bool)
        mask[np.triu_indices_from(mask)] = True
        # Set up the matplotlib figure
        f, ax = plt.subplots(figsize=(11, 9))
        # Generate a custom diverging colormap
        cmap = sns.diverging_palette(220, 10, as_cmap=True)
        # Draw the heatmap with the mask and correct aspect ratio
        sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3,square=True, xticklabels=5, yticklabels=5,linewidths=.5, cbar_kws={"shrink": .5}, ax=ax)
        plt.xticks(range(0,7),bandPower , color='black',rotation=40,horizontalalignment='right')
        plt.yticks(range(0,7),bandPower[::-1] , color='black',rotation=0,verticalalignment='bottom')
        plt.title(str(electrodes[j-1])+'_Experiment '+exp,horizontalalignment='center',verticalalignment='center',fontsize=20, color='blue')
        j= j+1
        figManager = plt.get_current_fig_manager()
        figManager.window.showMaximized()
        plt.show()


# Connectivity Heads

In [16]:
from __future__ import print_function
from __future__ import division
from __future__ import absolute_import

import scipy.interpolate
import numpy
import matplotlib 
import matplotlib.pyplot as plt

# close old plots
plt.close("all")

# some parameters
N = 300             # number of points for interpolation
xy_center = [2,2]   # center of the plot
radius = 2          # radius

# For k0:
for exp in experiments:

    SelectExpA = finalPatientsDF.loc[ (finalPatientsDF.experiment == exp) & (finalPatientsDF.cluster_Hierarchichal == 0)]
    for iBand in range( 0, len(bandPower) ):  
        corrMatrix = abs(SelectExpA.ix[:,np.arange(iBand,56,7)].corr())
        VExp=100*np.sum(corrMatrix, axis=1) /np.sum(np.sum(corrMatrix, axis=1) )  
        VExp.values
        m=0#np.amin(difEpx[0:56].values)
        ma=np.amax(VExp.values)
        meanRMap1 = [m, m]
        meanRMap2 = [m,  m,  m,  m , m, m, m, m, m, m, m, m, m]

        koordElectrodes = [[1.5,3.7],[2.5,3.7],[1.3,3],[2.7,3],[2,3],[1,2],[3,2],[2,2]]
        koordMap = [[1,4],[3,4],[1.5,3.7],[2.5,3.7],[1.3,3],[2.7,3],[2,3],[1,2],[3,2],[2,2],[1,1],[3,1],[2,1],[1,0],
                    [3,0],[0,3],[4,3],[0.6,2],[3.4,2],[0,1],[4,1],[1,3],[3,3]]
        #For map
        x,y = [],[]
        for i in koordMap:
            x.append(i[0])
            y.append(i[1])
        #For electrodes:    
        xElectrodes,yElectrodes = [],[]
        for i in koordElectrodes:
            xElectrodes.append(i[0])
            yElectrodes.append(i[1])  

        meanRMap=np.concatenate((meanRMap1,VExp,meanRMap2), axis=0)
        z = meanRMap
        xi = numpy.linspace(-2, 5, N)
        yi = numpy.linspace(-2, 5, N)
        zi = scipy.interpolate.griddata((x, y), z, (xi[None,:], yi[:,None]), method='cubic')

        # set points > radius to not-a-number. They will not be plotted.
        # the dr/2 makes the edges a bit smoother
        dr = xi[1] - xi[0]
        for i in range(N):
            for j in range(N):
                r = numpy.sqrt((xi[i] - xy_center[0])**2 + (yi[j] - xy_center[1])**2)
                if (r - dr/2) > radius:
                    zi[j,i] = "nan"

        # set aspect = 1 to make it a circle
        # make figure
        fig = plt.figure()
        ax = fig.add_subplot(111, aspect = 1)
        # draw a circle
        circle = matplotlib.patches.Circle(xy = xy_center, radius = radius, edgecolor = "k", facecolor = "darkblue")
        ax.add_patch(circle)
        # use different number of levels for the fill and the lines
        v = np.linspace(10, 14, 60, endpoint=True)
        CS = ax.contourf(xi, yi, zi, v, cmap = plt.cm.jet, zorder = 2)
        ax.contour(xi, yi, zi, 15, colors = "grey", zorder = 2)
        # make a color bar   
        cbar = fig.colorbar(CS, ax=ax)        
        # add the data points
        # I guess there are no data points outside the head...
        ax.scatter(xElectrodes, yElectrodes, marker = 'o', c = 'k', s = 30, zorder = 3)
        # make the axis invisible 
        for loc, spine in ax.spines.iteritems():
            # use ax.spines.items() in Python 3
            spine.set_linewidth(0)

        # remove the ticks
        ax.set_xticks([])
        ax.set_yticks([])

        # Add some body parts. Hide unwanted parts by setting the zorder low
        # add two ears
        circle = matplotlib.patches.Ellipse(xy = [0,2], width = 0.5, height = 1.0, angle = 0, edgecolor = "k", facecolor = "w", zorder = 0)
        ax.add_patch(circle)
        circle = matplotlib.patches.Ellipse(xy = [4,2], width = 0.5, height = 1.0, angle = 0, edgecolor = "k", facecolor = "w", zorder = 0)
        ax.add_patch(circle)
        # add a nose
        xy = [[1.5,3], [2,4.5],[2.5,3]]
        polygon = matplotlib.patches.Polygon(xy = xy, facecolor = "w", zorder = 0)
        ax.add_patch(polygon) 
        plt.title('Connectivity k0 subset '+ bandPower[iBand]+' Exp '+exp);   
        # set axes limits
        ax.set_xlim(-0.5, 4.5)
        ax.set_ylim(-0.5, 4.5)
        picPath = pathToSavePics + "k0Correlation% "+ bandPower[iBand]+ "Exp"+exp+".pdf"
        plt.savefig( picPath )
    plt.show() 
    

# For k1:
for exp in experiments:

    SelectExpA = finalPatientsDF.loc[ (finalPatientsDF.experiment == exp) & (finalPatientsDF.cluster_Hierarchichal == 1)]
    for iBand in range( 0, len(bandPower) ):  
        corrMatrix = abs(SelectExpA.ix[:,np.arange(iBand,56,7)].corr())
        VExp=100*np.sum(corrMatrix, axis=1) /np.sum(np.sum(corrMatrix, axis=1) )  
        VExp.values
        m=0#np.amin(difEpx[0:56].values)
        ma=np.amax(VExp.values)
        meanRMap1 = [m, m]
        meanRMap2 = [m,  m,  m,  m , m, m, m, m, m, m, m, m, m]

        koordElectrodes = [[1.5,3.7],[2.5,3.7],[1.3,3],[2.7,3],[2,3],[1,2],[3,2],[2,2]]
        koordMap = [[1,4],[3,4],[1.5,3.7],[2.5,3.7],[1.3,3],[2.7,3],[2,3],[1,2],[3,2],[2,2],[1,1],[3,1],[2,1],[1,0],
                    [3,0],[0,3],[4,3],[0.6,2],[3.4,2],[0,1],[4,1],[1,3],[3,3]]
        #For map
        x,y = [],[]
        for i in koordMap:
            x.append(i[0])
            y.append(i[1])
        #For electrodes:    
        xElectrodes,yElectrodes = [],[]
        for i in koordElectrodes:
            xElectrodes.append(i[0])
            yElectrodes.append(i[1])  

        meanRMap=np.concatenate((meanRMap1,VExp,meanRMap2), axis=0)
        z = meanRMap
        xi = numpy.linspace(-2, 5, N)
        yi = numpy.linspace(-2, 5, N)
        zi = scipy.interpolate.griddata((x, y), z, (xi[None,:], yi[:,None]), method='cubic')

        # set points > radius to not-a-number. They will not be plotted.
        # the dr/2 makes the edges a bit smoother
        dr = xi[1] - xi[0]
        for i in range(N):
            for j in range(N):
                r = numpy.sqrt((xi[i] - xy_center[0])**2 + (yi[j] - xy_center[1])**2)
                if (r - dr/2) > radius:
                    zi[j,i] = "nan"

        # set aspect = 1 to make it a circle
        # make figure
        fig = plt.figure()
        ax = fig.add_subplot(111, aspect = 1)
        # draw a circle
        circle = matplotlib.patches.Circle(xy = xy_center, radius = radius, edgecolor = "k", facecolor = "darkblue")
        ax.add_patch(circle)
        # use different number of levels for the fill and the lines
        v = np.linspace(10, 14, 60, endpoint=True)
        CS = ax.contourf(xi, yi, zi, v, cmap = plt.cm.jet, zorder = 2)
        ax.contour(xi, yi, zi, 15, colors = "grey", zorder = 2)
        # make a color bar   
        cbar = fig.colorbar(CS, ax=ax)        
        # add the data points
        # I guess there are no data points outside the head...
        ax.scatter(xElectrodes, yElectrodes, marker = 'o', c = 'k', s = 30, zorder = 3)
        # make the axis invisible 
        for loc, spine in ax.spines.iteritems():
            # use ax.spines.items() in Python 3
            spine.set_linewidth(0)

        # remove the ticks
        ax.set_xticks([])
        ax.set_yticks([])

        # Add some body parts. Hide unwanted parts by setting the zorder low
        # add two ears
        circle = matplotlib.patches.Ellipse(xy = [0,2], width = 0.5, height = 1.0, angle = 0, edgecolor = "k", facecolor = "w", zorder = 0)
        ax.add_patch(circle)
        circle = matplotlib.patches.Ellipse(xy = [4,2], width = 0.5, height = 1.0, angle = 0, edgecolor = "k", facecolor = "w", zorder = 0)
        ax.add_patch(circle)
        # add a nose
        xy = [[1.5,3], [2,4.5],[2.5,3]]
        polygon = matplotlib.patches.Polygon(xy = xy, facecolor = "w", zorder = 0)
        ax.add_patch(polygon) 
        plt.title('Connectivity k1 subset '+ bandPower[iBand]+' Exp '+exp);   
        # set axes limits
        ax.set_xlim(-0.5, 4.5)
        ax.set_ylim(-0.5, 4.5)
        picPath = pathToSavePics + "k1Correlation% "+ bandPower[iBand]+ "Exp"+exp+".pdf"
        plt.savefig( picPath )
    plt.show() 